In [1]:
# import libraries
import pandas as pd
import geopandas as gpd
import numpy as np

In [2]:
# load knife crime data
kc_df = pd.read_excel('kc_regional_variation.xlsx', header=[0,1])
kc_df.head()

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0 2008/09  \
  Unnamed: 0_level_1 Unnamed: 1_level_1 Unnamed: 2_level_1  Number   
0                NaN          Cleveland                NaN   207.0   
1                NaN             Durham                NaN   152.0   
2                NaN        Northumbria                NaN   470.0   
3                NaN        North East                 NaN   829.0   
4                NaN                NaN                NaN     NaN   

                                                                      2009/10  \
  Offences per 100,000 population  Offences per 100,000 population .1  Number   
0                        36.904974                                NaN   227.0   
1                        24.394158                                NaN   118.0   
2                        32.759462                                NaN   401.0   
3                        31.656929                                NaN   746.0   
4                              NaN                                NaN     NaN   

                                                                      2010/11  \
  Offences per 100,000 population  Offences per 100,000 population .1  Number   
0                        40.470672                                NaN   297.0   
1                         18.93757                                NaN   159.0   
2                        27.950094                                NaN   393.0   
3                        28.487417                                NaN   849.0   
4                              NaN                                NaN     NaN   

   ...                          2018/19                                     \
   ... Offences per 100,000 population  Offences per 100,000 population .1   
0  ...                       105.605342                                NaN   
1  ...                        30.774062                                NaN   
2  ...                        64.348658                                NaN   
3  ...                        65.133526                                NaN   
4  ...                              NaN                                NaN   

  2019/20                                                                      \
   Number Offences per 100,000 population  Offences per 100,000 population .1   
0   708.0                       124.709803                                NaN   
1   165.0                        26.043886                                NaN   
2   794.0                         54.50882                                NaN   
3  1667.0                        62.718475                                NaN   
4     NaN                              NaN                                NaN   

  2020/21                                                                      \
   Number Offences per 100,000 population7 Offences per 100,000 population7.1   
0   692.0                       121.586742                                NaN   
1   138.0                        21.667554                                NaN   
2   857.0                         58.54213                                NaN   
3  1687.0                               63                                NaN   
4     NaN                              NaN                                NaN   

   Change per 100,000, 2010/11 - 2020/21            
                          Actual change   % change  
0                              68.303642  1.281901  
1                              -4.035647  -0.15701  
2                              30.873868  1.115859  
3                              30.300878  0.926657  
4                                    NaN       NaN  

[5 rows x 44 columns]

In [3]:
# reorg the columns and drop columns with no data
kc_df.columns = ['_'.join(col).strip() for col in kc_df.columns.values]
kc_df = kc_df.dropna(how='all').dropna(axis=1, how='all').dropna(axis=0, how='all')
kc_df.head()

,Unnamed: 1_level_0_Unnamed: 1_level_1,Unnamed: 2_level_0_Unnamed: 2_level_1,2008/09_Number,"2008/09_Offences per 100,000 population",2009/10_Number,"2009/10_Offences per 100,000 population",2010/11_Number,"2010/11_Offences per 100,000 population","2010/11_Offences per 100,000 population .1",2011/12_Number,...,2017/18_Number,"2017/18_Offences per 100,000 population",2018/19_Number,"2018/19_Offences per 100,000 population",2019/20_Number,"2019/20_Offences per 100,000 population",2020/21_Number,"2020/21_Offences per 100,000 population7","Change per 100,000, 2010/11 - 2020/21_Actual change","Change per 100,000, 2010/11 - 2020/21_% change"
0,Cleveland,NaN,207.0,36.904974,227.0,40.470672,297.0,53.2831,NaN,275.0,...,620.0,109.212612,601.0,105.605342,708.0,124.709803,692.0,121.586742,68.303642,1.281901
1,Durham,NaN,152.0,24.394158,118.0,18.93757,159.0,25.703201,NaN,157.0,...,264.0,41.673244,196.0,30.774062,165.0,26.043886,138.0,21.667554,-4.035647,-0.15701
2,Northumbria,NaN,470.0,32.759462,401.0,27.950094,393.0,27.668262,NaN,375.0,...,812.0,55.746258,942.0,64.348658,794.0,54.50882,857.0,58.54213,30.873868,1.115859
3,North East,NaN,829.0,31.656929,746.0,28.487417,849.0,32.699122,NaN,807.0,...,1696.0,63.812175,1739.0,65.133526,1667.0,62.718475,1687.0,63,30.300878,0.926657
5,Cheshire,NaN,341.0,32.813703,276.0,26.558891,266.0,25.857879,NaN,234.0,...,378.0,35.683942,374.0,35.064692,401.0,37.761819,265.0,24.844208,-1.013671,-0.039202


In [4]:
# rename the name columns and drop upexpected rows
kc_df = kc_df.rename(columns={'Unnamed: 1_level_0_Unnamed: 1_level_1': 'Name'}).drop('Unnamed: 2_level_0_Unnamed: 2_level_1', axis=1)
kc_df = kc_df[kc_df['Name'].notna()]
kc_df.columns

Index(['Name', '2008/09_Number', '2008/09_Offences per 100,000 population',
       '2009/10_Number', '2009/10_Offences per 100,000 population',
       '2010/11_Number', '2010/11_Offences per 100,000 population',
       '2010/11_Offences per 100,000 population .1', '2011/12_Number',
       '2011/12_Offences per 100,000 population',
       '2011/12_Offences per 100,000 population .1', '2012/13_Number',
       '2012/13_Offences per 100,000 population',
       '2012/13_Offences per 100,000 population .1', '2013/14_Number',
       '2013/14_Offences per 100,000 population',
       '2013/14_Offences per 100,000 population .1', '2014/15_Number',
       '2014/15_Offences per 100,000 population', '2015/16_Number',
       '2015/16_Offences per 100,000 population', '2016/17_Number',
       '2016/17_Offences per 100,000 population',
       '2016/17_Offences per 100,000 population .1', '2017/18_Number',
       '2017/18_Offences per 100,000 population', '2018/19_Number',
       '2018/19_Offences per 

In [5]:
# drop upexpected columns
kc_df.drop(list(kc_df.filter(regex = 'Offences per 100,000 population .1')), axis = 1, inplace=True)

# rename column
kc_df.rename(columns={'2020/21_Offences per 100,000 population7': '2020/21_Offences per 100,000 population'}, inplace=True)
kc_df.head()

,Name,2008/09_Number,"2008/09_Offences per 100,000 population",2009/10_Number,"2009/10_Offences per 100,000 population",2010/11_Number,"2010/11_Offences per 100,000 population",2011/12_Number,"2011/12_Offences per 100,000 population",2012/13_Number,...,2017/18_Number,"2017/18_Offences per 100,000 population",2018/19_Number,"2018/19_Offences per 100,000 population",2019/20_Number,"2019/20_Offences per 100,000 population",2020/21_Number,"2020/21_Offences per 100,000 population","Change per 100,000, 2010/11 - 2020/21_Actual change","Change per 100,000, 2010/11 - 2020/21_% change"
0,Cleveland,207.0,36.904974,227.0,40.470672,297.0,53.2831,275.0,49.239033,270.0,...,620.0,109.212612,601.0,105.605342,708.0,124.709803,692.0,121.586742,68.303642,1.281901
1,Durham,152.0,24.394158,118.0,18.93757,159.0,25.703201,157.0,25.330752,140.0,...,264.0,41.673244,196.0,30.774062,165.0,26.043886,138.0,21.667554,-4.035647,-0.15701
2,Northumbria,470.0,32.759462,401.0,27.950094,393.0,27.668262,375.0,26.330572,316.0,...,812.0,55.746258,942.0,64.348658,794.0,54.50882,857.0,58.54213,30.873868,1.115859
3,North East,829.0,31.656929,746.0,28.487417,849.0,32.699122,807.0,31.009837,726.0,...,1696.0,63.812175,1739.0,65.133526,1667.0,62.718475,1687.0,63,30.300878,0.926657
5,Cheshire,341.0,32.813703,276.0,26.558891,266.0,25.857879,234.0,22.670025,205.0,...,378.0,35.683942,374.0,35.064692,401.0,37.761819,265.0,24.844208,-1.013671,-0.039202


In [9]:
kc_df['Name'].values

array(['Cleveland', 'Durham', 'Northumbria', 'North East ', 'Cheshire',
       'Cumbria', 'Greater Manchester4', 'Lancashire', 'Merseyside',
       'North West, including GMP', 'North West, Less GMP', 'Humberside',
       'North Yorkshire', 'South Yorkshire', 'West Yorkshire',
       'Yorkshire and the Humber ', 'Derbyshire', 'Leicestershire',
       'Lincolnshire', 'Northamptonshire', 'Nottinghamshire',
       'East Midlands ', 'Staffordshire', 'Warwickshire', 'West Mercia',
       'West Midlands', 'West Midlands Region', 'Bedfordshire',
       'Cambridgeshire', 'Essex', 'Hertfordshire', 'Norfolk', 'Suffolk',
       'East of England ', 'City of London', 'Metropolitan Police',
       'London', 'Hampshire', 'Kent', 'Surrey5', 'Sussex',
       'Thames Valley', 'South East ', 'Avon and Somerset',
       'Devon and Cornwall', 'Dorset', 'Gloucestershire', 'Wiltshire',
       'South West ', 'Dyfed-Powys', 'Gwent', 'North Wales',
       'South Wales', 'WALES', 'British Transport Police',
    

In [10]:
# Some rows have names that are uncommon (contains space, number, etc.)
area_name = kc_df['Name'].values.copy()
area_name = np.array([name.strip() for name in area_name])

for ind, name in enumerate(area_name):
    discard = name.find(', including GMP')
    if discard != -1:
        area_name[ind] = name[:discard]

area_name

array(['Cleveland', 'Durham', 'Northumbria', 'North East', 'Cheshire',
       'Cumbria', 'Greater Manchester4', 'Lancashire', 'Merseyside',
       'North West', 'North West, Less GMP', 'Humberside',
       'North Yorkshire', 'South Yorkshire', 'West Yorkshire',
       'Yorkshire and the Humber', 'Derbyshire', 'Leicestershire',
       'Lincolnshire', 'Northamptonshire', 'Nottinghamshire',
       'East Midlands', 'Staffordshire', 'Warwickshire', 'West Mercia',
       'West Midlands', 'West Midlands Region', 'Bedfordshire',
       'Cambridgeshire', 'Essex', 'Hertfordshire', 'Norfolk', 'Suffolk',
       'East of England', 'City of London', 'Metropolitan Police',
       'London', 'Hampshire', 'Kent', 'Surrey5', 'Sussex',
       'Thames Valley', 'South East', 'Avon and Somerset',
       'Devon and Cornwall', 'Dorset', 'Gloucestershire', 'Wiltshire',
       'South West', 'Dyfed-Powys', 'Gwent', 'North Wales', 'South Wales',
       'WALES', 'British Transport Police', 'ENGLAND AND WALES',
    

In [11]:
# replace the name column with the cleaned names
kc_df['Name'] = area_name

# fix the rest 2 special case
kc_df.loc[kc_df['Name']=='WALES','Name'] = 'Wales'
kc_df.loc[kc_df['Name']=='Surrey5','Name'] = 'Surrey'

kc_df['Name'].values

array(['Cleveland', 'Durham', 'Northumbria', 'North East', 'Cheshire',
       'Cumbria', 'Greater Manchester4', 'Lancashire', 'Merseyside',
       'North West', 'North West, Less GMP', 'Humberside',
       'North Yorkshire', 'South Yorkshire', 'West Yorkshire',
       'Yorkshire and the Humber', 'Derbyshire', 'Leicestershire',
       'Lincolnshire', 'Northamptonshire', 'Nottinghamshire',
       'East Midlands', 'Staffordshire', 'Warwickshire', 'West Mercia',
       'West Midlands', 'West Midlands Region', 'Bedfordshire',
       'Cambridgeshire', 'Essex', 'Hertfordshire', 'Norfolk', 'Suffolk',
       'East of England', 'City of London', 'Metropolitan Police',
       'London', 'Hampshire', 'Kent', 'Surrey', 'Sussex', 'Thames Valley',
       'South East', 'Avon and Somerset', 'Devon and Cornwall', 'Dorset',
       'Gloucestershire', 'Wiltshire', 'South West', 'Dyfed-Powys',
       'Gwent', 'North Wales', 'South Wales', 'Wales',
       'British Transport Police', 'ENGLAND AND WALES',
     

In [12]:
from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile

# download the uk regions boundaries shapefile
url = 'https://opendata.arcgis.com/api/v3/datasets/01fd6b2d7600446d8af768005992f76a_0/downloads/data?format=shp&spatialRefId=27700&where=1%3D1'

with urlopen(url) as zipresp:
    with ZipFile(BytesIO(zipresp.read())) as zfile:
        zfile.extractall('uk_regions_boundaries')

In [13]:
# load the shapefile
uk_regions = gpd.read_file('uk_regions_boundaries/NUTS_Level_1_(January_2018)_Boundaries.shp')
uk_regions.head()

,objectid,nuts118cd,nuts118nm,bng_e,bng_n,long,lat,st_areasha,st_lengths,geometry
0,1,UKC,North East (England),417313,600358,-1.72890,55.297031,8.592429e+09,1.008435e+06,"MULTIPOLYGON (((398583.804 653093.703, 398566...."
1,2,UKD,North West (England),350015,506280,-2.77237,54.449451,1.416409e+10,2.075324e+06,"MULTIPOLYGON (((357067.497 586765.599, 357029...."
2,3,UKE,Yorkshire and The Humber,446903,448736,-1.28712,53.932640,1.540912e+10,1.385386e+06,"MULTIPOLYGON (((478300.699 518863.096, 478309...."
3,4,UKF,East Midlands (England),477660,322635,-0.84967,52.795719,1.564327e+10,1.332317e+06,"MULTIPOLYGON (((512572.598 414469.499, 512619...."
4,5,UKG,West Midlands (England),386294,295477,-2.20358,52.556969,1.300374e+10,9.539014e+05,"POLYGON ((402840.498 368845.598, 402862.397 36..."


In [14]:
# we only need this two columns
uk_regions = uk_regions[['nuts118nm', 'geometry']]

# insepct the data
uk_regions

,nuts118nm,geometry
0,North East (England),"MULTIPOLYGON (((398583.804 653093.703, 398566...."
1,North West (England),"MULTIPOLYGON (((357067.497 586765.599, 357029...."
2,Yorkshire and The Humber,"MULTIPOLYGON (((478300.699 518863.096, 478309...."
3,East Midlands (England),"MULTIPOLYGON (((512572.598 414469.499, 512619...."
4,West Midlands (England),"POLYGON ((402840.498 368845.598, 402862.397 36..."
5,East of England,"MULTIPOLYGON (((600880.500 346519.099, 600924...."
6,London,"MULTIPOLYGON (((531643.597 200802.700, 531658...."
7,South East (England),"MULTIPOLYGON (((490173.603 256107.999, 490234...."
8,South West (England),"MULTIPOLYGON (((416361.796 245471.004, 416367...."
9,Wales,"MULTIPOLYGON (((312098.398 385255.901, 312105...."


In [15]:
# The name columns contain unexpected suffix and we need to remove them in order to merge the two dataframe
area_name = uk_regions['nuts118nm'].values.copy()
for ind, name in enumerate(area_name):
    discard = name.find(' (England)')
    if discard != -1:
        area_name[ind] = name[:discard]
area_name = np.array([name.strip() for name in area_name])
uk_regions['Area Name'] = area_name
uk_regions

,nuts118nm,geometry,Area Name
0,North East (England),"MULTIPOLYGON (((398583.804 653093.703, 398566....",North East
1,North West (England),"MULTIPOLYGON (((357067.497 586765.599, 357029....",North West
2,Yorkshire and The Humber,"MULTIPOLYGON (((478300.699 518863.096, 478309....",Yorkshire and The Humber
3,East Midlands (England),"MULTIPOLYGON (((512572.598 414469.499, 512619....",East Midlands
4,West Midlands (England),"POLYGON ((402840.498 368845.598, 402862.397 36...",West Midlands
5,East of England,"MULTIPOLYGON (((600880.500 346519.099, 600924....",East of England
6,London,"MULTIPOLYGON (((531643.597 200802.700, 531658....",London
7,South East (England),"MULTIPOLYGON (((490173.603 256107.999, 490234....",South East
8,South West (England),"MULTIPOLYGON (((416361.796 245471.004, 416367....",South West
9,Wales,"MULTIPOLYGON (((312098.398 385255.901, 312105....",Wales


In [16]:
kc_df['Name'].values

array(['Cleveland', 'Durham', 'Northumbria', 'North East', 'Cheshire',
       'Cumbria', 'Greater Manchester4', 'Lancashire', 'Merseyside',
       'North West', 'North West, Less GMP', 'Humberside',
       'North Yorkshire', 'South Yorkshire', 'West Yorkshire',
       'Yorkshire and the Humber', 'Derbyshire', 'Leicestershire',
       'Lincolnshire', 'Northamptonshire', 'Nottinghamshire',
       'East Midlands', 'Staffordshire', 'Warwickshire', 'West Mercia',
       'West Midlands', 'West Midlands Region', 'Bedfordshire',
       'Cambridgeshire', 'Essex', 'Hertfordshire', 'Norfolk', 'Suffolk',
       'East of England', 'City of London', 'Metropolitan Police',
       'London', 'Hampshire', 'Kent', 'Surrey', 'Sussex', 'Thames Valley',
       'South East', 'Avon and Somerset', 'Devon and Cornwall', 'Dorset',
       'Gloucestershire', 'Wiltshire', 'South West', 'Dyfed-Powys',
       'Gwent', 'North Wales', 'South Wales', 'Wales',
       'British Transport Police', 'ENGLAND AND WALES',
     

In [17]:
# The west midlands is named 'West Midlands Region' in the knife crime dataframe and we need to make them consistent.
uk_regions.loc[uk_regions['Area Name']=='West Midlands', 'Area Name'] = 'West Midlands Region'

In [18]:
# merge the two dataframes by their region names
uk_regions_with_kc = pd.merge(uk_regions, kc_df, 
                              left_on=uk_regions['Area Name'].str.lower(), 
                              right_on=kc_df['Name'].str.lower(), 
                              how='left')
uk_regions_with_kc

,key_0,nuts118nm,geometry,Area Name,Name,2008/09_Number,"2008/09_Offences per 100,000 population",2009/10_Number,"2009/10_Offences per 100,000 population",2010/11_Number,...,2017/18_Number,"2017/18_Offences per 100,000 population",2018/19_Number,"2018/19_Offences per 100,000 population",2019/20_Number,"2019/20_Offences per 100,000 population",2020/21_Number,"2020/21_Offences per 100,000 population","Change per 100,000, 2010/11 - 2020/21_Actual change","Change per 100,000, 2010/11 - 2020/21_% change"
0,north east,North East (England),"MULTIPOLYGON (((398583.804 653093.703, 398566....",North East,North East,829.0,31.656929,746.0,28.487417,849.0,...,1696.0,63.812175,1739.0,65.133526,1667.0,62.718475,1687.0,63,30.300878,0.926657
1,north west,North West (England),"MULTIPOLYGON (((357067.497 586765.599, 357029....",North West,North West,4998.0,70.067712,4163.0,58.361722,3722.0,...,4289.0,58.778788,6255.0,85.205214,6353.0,87,5726.0,78,25.250567,0.478689
2,yorkshire and the humber,Yorkshire and The Humber,"MULTIPOLYGON (((478300.699 518863.096, 478309....",Yorkshire and The Humber,Yorkshire and the Humber,2515.0,46.921642,2446.0,45.634328,2950.0,...,5384.0,98.255347,5758.0,104.633836,5352.0,97.671096,4576.0,83,27.215423,0.487866
3,east midlands,East Midlands (England),"MULTIPOLYGON (((512572.598 414469.499, 512619....",East Midlands,East Midlands,2308.0,49.768194,2001.0,43.148248,2000.0,...,2930.0,60.989571,3541.0,73.223185,3739.0,77.82856,3375.0,70,25.921894,0.58809
4,west midlands region,West Midlands (England),"POLYGON ((402840.498 368845.598, 402862.397 36...",West Midlands Region,West Midlands Region,4701.0,82.281694,4130.0,72.28747,5328.0,...,5562.0,94.258406,6430.0,108.358611,6698.0,114,6070.0,102,7.004725,0.073738
5,east of england,East of England,"MULTIPOLYGON (((600880.500 346519.099, 600924....",East of England,East of England,2097.0,34.842569,1936.0,32.167484,1901.0,...,3507.0,56.552658,3104.0,49.773901,3979.0,64.164855,4155.0,67,34.573008,1.06618
6,london,London,"MULTIPOLYGON (((531643.597 200802.700, 531658....",London,London,12259.0,NaN,12570.0,NaN,12501.0,...,13734.0,154.174291,13879.0,154.864985,14716.0,165.198318,10150.0,113,-39.36946,-0.258382
7,south east,South East (England),"MULTIPOLYGON (((490173.603 256107.999, 490234....",South East,South East,3224.0,36.331673,2896.0,32.635399,2358.0,...,4207.0,46.060699,4658.0,50.740188,4684.0,51.283034,4070.0,49,21.748706,0.79808
8,south west,South West (England),"MULTIPOLYGON (((416361.796 245471.004, 416367....",South West,South West,1875.0,34.573683,1680.0,30.97802,1902.0,...,2113.0,37.732817,2722.0,48.394553,2987.0,53.34181,2740.0,49,13.118624,0.365611
9,wales,Wales,"MULTIPOLYGON (((312098.398 385255.901, 312105....",Wales,Wales,928.0,30.012937,765.0,24.741268,700.0,...,1072.0,34.154268,1237.0,39.233721,1477.0,47.058734,1461.0,61,38.152556,1.669883


In [19]:
# since we only care about England and Wales, we can drop the rows for Scotland and NI.
uk_regions_with_kc.drop([10,11], inplace=True)
# also drop unnecessary columns
uk_regions_with_kc.drop(['key_0', 'Name'], axis=1, inplace=True)

In [20]:
# inspect data types for each column
uk_regions_with_kc.dtypes

nuts118nm                                                object
geometry                                               geometry
Area Name                                                object
2008/09_Number                                          float64
2008/09_Offences per 100,000 population                  object
2009/10_Number                                          float64
2009/10_Offences per 100,000 population                  object
2010/11_Number                                          float64
2010/11_Offences per 100,000 population                  object
2011/12_Number                                          float64
2011/12_Offences per 100,000 population                  object
2012/13_Number                                          float64
2012/13_Offences per 100,000 population                  object
2013/14_Number                                          float64
2013/14_Offences per 100,000 population                  object
2014/15_Number                          

In [21]:
# some columns have the wrong data types, we need to correct them.
# columns that should be integer type
int_col = [uk_regions_with_kc.columns[i*2+1] for i in range(1,len(uk_regions_with_kc.columns)//2)]
# columns that should be float (decimal) type
float_col = [uk_regions_with_kc.columns[i*2+2] for i in range(1,len(uk_regions_with_kc.columns)//2)]

# change their data types accordingly
uk_regions_with_kc[int_col] = uk_regions_with_kc[int_col].astype('int')
uk_regions_with_kc[float_col] = uk_regions_with_kc[float_col].astype('float')
uk_regions_with_kc.dtypes

nuts118nm                                                object
geometry                                               geometry
Area Name                                                object
2008/09_Number                                            int32
2008/09_Offences per 100,000 population                 float64
2009/10_Number                                            int32
2009/10_Offences per 100,000 population                 float64
2010/11_Number                                            int32
2010/11_Offences per 100,000 population                 float64
2011/12_Number                                            int32
2011/12_Offences per 100,000 population                 float64
2012/13_Number                                            int32
2012/13_Offences per 100,000 population                 float64
2013/14_Number                                            int32
2013/14_Offences per 100,000 population                 float64
2014/15_Number                          

In [22]:
# unflatten the data for visualisation purpose (transfer from wide data to long data)
unflatten_uk_regions_kc_int = pd.melt(uk_regions_with_kc[['nuts118nm', 'geometry', 'Area Name']+int_col], 
                                    id_vars=['nuts118nm', 'geometry', 'Area Name'],
                                    var_name='Year', value_vars=int_col, value_name='Number')
unflatten_uk_regions_kc_float = pd.melt(uk_regions_with_kc[['nuts118nm', 'geometry', 'Area Name']+float_col], 
                                      id_vars=['nuts118nm', 'geometry', 'Area Name'],
                                      var_name='Year', value_vars=float_col, value_name='Perc_per_pop')

In [23]:
# change their year columns to the actual year they are representing
unflatten_uk_regions_kc_int['Year'] = unflatten_uk_regions_kc_int['Year'].apply(int_col.index)
unflatten_uk_regions_kc_float['Year'] = unflatten_uk_regions_kc_float['Year'].apply(float_col.index)

In [24]:
# combine the two data frame for different types of data
unflatten_uk_regions_kc = pd.merge(unflatten_uk_regions_kc_int, unflatten_uk_regions_kc_float, how='inner')
unflatten_uk_regions_kc['Year'] = unflatten_uk_regions_kc['Year'] + 2008
unflatten_uk_regions_kc

,nuts118nm,geometry,Area Name,Year,Number,Perc_per_pop
0,North East (England),"MULTIPOLYGON (((398583.804 653093.703, 398566....",North East,2008,829,31.656929
1,North West (England),"MULTIPOLYGON (((357067.497 586765.599, 357029....",North West,2008,4998,70.067712
2,Yorkshire and The Humber,"MULTIPOLYGON (((478300.699 518863.096, 478309....",Yorkshire and The Humber,2008,2515,46.921642
3,East Midlands (England),"MULTIPOLYGON (((512572.598 414469.499, 512619....",East Midlands,2008,2308,49.768194
4,West Midlands (England),"POLYGON ((402840.498 368845.598, 402862.397 36...",West Midlands Region,2008,4701,82.281694
...,...,...,...,...,...,...
135,East of England,"MULTIPOLYGON (((600880.500 346519.099, 600924....",East of England,2021,34,1.066180
136,London,"MULTIPOLYGON (((531643.597 200802.700, 531658....",London,2021,-39,-0.258382
137,South East (England),"MULTIPOLYGON (((490173.603 256107.999, 490234....",South East,2021,21,0.798080
138,South West (England),"MULTIPOLYGON (((416361.796 245471.004, 416367....",South West,2021,13,0.365611


In [26]:
# the rows with 'Year' equals 2021 actually represents the total change of knife crimes from 2010 to 2021, hence we need to seperate it
# from the rest of the data because they represent different things
chg_uk_regions = unflatten_uk_regions_kc[unflatten_uk_regions_kc['Year']==2021].reset_index(drop=True).copy()
chg_uk_regions.rename(columns={'Number': 'Act_chg_per_pop', 'Perc_per_pop': 'Perc_chg_per_pop'}, inplace=True)
chg_uk_regions.drop('Year', axis=1, inplace=True)
chg_uk_regions

,nuts118nm,geometry,Area Name,Act_chg_per_pop,Perc_chg_per_pop
0,North East (England),"MULTIPOLYGON (((398583.804 653093.703, 398566....",North East,30,0.926657
1,North West (England),"MULTIPOLYGON (((357067.497 586765.599, 357029....",North West,25,0.478689
2,Yorkshire and The Humber,"MULTIPOLYGON (((478300.699 518863.096, 478309....",Yorkshire and The Humber,27,0.487866
3,East Midlands (England),"MULTIPOLYGON (((512572.598 414469.499, 512619....",East Midlands,25,0.588090
4,West Midlands (England),"POLYGON ((402840.498 368845.598, 402862.397 36...",West Midlands Region,7,0.073738
5,East of England,"MULTIPOLYGON (((600880.500 346519.099, 600924....",East of England,34,1.066180
6,London,"MULTIPOLYGON (((531643.597 200802.700, 531658....",London,-39,-0.258382
7,South East (England),"MULTIPOLYGON (((490173.603 256107.999, 490234....",South East,21,0.798080
8,South West (England),"MULTIPOLYGON (((416361.796 245471.004, 416367....",South West,13,0.365611
9,Wales,"MULTIPOLYGON (((312098.398 385255.901, 312105....",Wales,38,1.669883


In [28]:
annual_uk_regions = unflatten_uk_regions_kc[unflatten_uk_regions_kc['Year']!=2021].copy()
annual_uk_regions.rename(columns={'Perc_per_pop': 'Number_per_pop'}, inplace=True)
annual_uk_regions

,nuts118nm,geometry,Area Name,Year,Number,Number_per_pop
0,North East (England),"MULTIPOLYGON (((398583.804 653093.703, 398566....",North East,2008,829,31.656929
1,North West (England),"MULTIPOLYGON (((357067.497 586765.599, 357029....",North West,2008,4998,70.067712
2,Yorkshire and The Humber,"MULTIPOLYGON (((478300.699 518863.096, 478309....",Yorkshire and The Humber,2008,2515,46.921642
3,East Midlands (England),"MULTIPOLYGON (((512572.598 414469.499, 512619....",East Midlands,2008,2308,49.768194
4,West Midlands (England),"POLYGON ((402840.498 368845.598, 402862.397 36...",West Midlands Region,2008,4701,82.281694
...,...,...,...,...,...,...
125,East of England,"MULTIPOLYGON (((600880.500 346519.099, 600924....",East of England,2020,4155,67.000000
126,London,"MULTIPOLYGON (((531643.597 200802.700, 531658....",London,2020,10150,113.000000
127,South East (England),"MULTIPOLYGON (((490173.603 256107.999, 490234....",South East,2020,4070,49.000000
128,South West (England),"MULTIPOLYGON (((416361.796 245471.004, 416367....",South West,2020,2740,49.000000


In [29]:
annual_uk_regions.to_file('uk_regions_with_kc/uk_regions_with_kc.shp')
chg_uk_regions.to_file('uk_regions_with_kc/chg_uk_regions_with_kc.shp')

C:\Users\Yulun\AppData\Local\Temp\ipykernel_11888\438910612.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  annual_uk_regions.to_file('uk_regions_with_kc/uk_regions_with_kc.shp')
C:\Users\Yulun\AppData\Local\Temp\ipykernel_11888\438910612.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  chg_uk_regions.to_file('uk_regions_with_kc/chg_uk_regions_with_kc.shp')


In [30]:
# do the same for the pfa data
url = 'https://opendata.arcgis.com/api/v3/datasets/1c8b832d08eb4f77a453156a29f4b86b_0/downloads/data?format=shp&spatialRefId=27700&where=1%3D1'

with urlopen(url) as zipresp:
    with ZipFile(BytesIO(zipresp.read())) as zfile:
        zfile.extractall('police_areas_boundaries')
        
police_areas = gpd.read_file('police_areas_boundaries/Police_Force_Areas_Dec_2016_FCB_in_England_and_Wales.shp')
police_areas.head()

,OBJECTID,pfa16cd,pfa16nm,bng_e,bng_n,long,lat,GlobalID,SHAPE_Leng,SHAPE_Area,geometry
0,1,E23000001,Metropolitan Police,517516,178392,-0.30864,51.492271,{3B0F53A6-9B0B-4F15-BC08-A5884C5C769C},424522.342920,1.570606e+09,"MULTIPOLYGON (((531643.597 200802.700, 531658...."
1,2,E23000002,Cumbria,341868,527389,-2.90218,54.638260,{0D81F3AD-5EDA-47AE-B02A-FEB81EB45094},987067.168716,6.822195e+09,"MULTIPOLYGON (((357067.497 586765.599, 357029...."
2,3,E23000003,Lancashire,363669,441026,-2.55392,53.864182,{435DE934-AD9C-4F8A-9CB7-4545F04A4158},795005.042947,3.076413e+09,"MULTIPOLYGON (((370171.702 482434.604, 370185...."
3,4,E23000004,Merseyside,344324,390164,-2.83893,53.405350,{A9457993-3013-4BB8-A235-AF5C291D1A54},280940.941577,6.467317e+08,"MULTIPOLYGON (((336272.101 422872.103, 336303...."
4,5,E23000005,Greater Manchester,377940,401102,-2.33407,53.506149,{B392B419-9206-48F4-8011-4C757BD9B3E6},248705.576852,1.275981e+09,"POLYGON ((391363.101 420849.902, 391421.401 42..."


In [31]:
kc_df['Name'].values

array(['Cleveland', 'Durham', 'Northumbria', 'North East', 'Cheshire',
       'Cumbria', 'Greater Manchester4', 'Lancashire', 'Merseyside',
       'North West', 'North West, Less GMP', 'Humberside',
       'North Yorkshire', 'South Yorkshire', 'West Yorkshire',
       'Yorkshire and the Humber', 'Derbyshire', 'Leicestershire',
       'Lincolnshire', 'Northamptonshire', 'Nottinghamshire',
       'East Midlands', 'Staffordshire', 'Warwickshire', 'West Mercia',
       'West Midlands', 'West Midlands Region', 'Bedfordshire',
       'Cambridgeshire', 'Essex', 'Hertfordshire', 'Norfolk', 'Suffolk',
       'East of England', 'City of London', 'Metropolitan Police',
       'London', 'Hampshire', 'Kent', 'Surrey', 'Sussex', 'Thames Valley',
       'South East', 'Avon and Somerset', 'Devon and Cornwall', 'Dorset',
       'Gloucestershire', 'Wiltshire', 'South West', 'Dyfed-Powys',
       'Gwent', 'North Wales', 'South Wales', 'Wales',
       'British Transport Police', 'ENGLAND AND WALES',
     

In [32]:
police_areas = police_areas[['pfa16nm', 'geometry']]
kc_df.loc[kc_df['Name']=='Greater Manchester4', 'Name'] = 'Greater Manchester'
police_areas_with_kc = pd.merge(police_areas, kc_df, left_on='pfa16nm', right_on='Name', how='left')
police_areas_with_kc

,pfa16nm,geometry,Name,2008/09_Number,"2008/09_Offences per 100,000 population",2009/10_Number,"2009/10_Offences per 100,000 population",2010/11_Number,"2010/11_Offences per 100,000 population",2011/12_Number,...,2017/18_Number,"2017/18_Offences per 100,000 population",2018/19_Number,"2018/19_Offences per 100,000 population",2019/20_Number,"2019/20_Offences per 100,000 population",2020/21_Number,"2020/21_Offences per 100,000 population","Change per 100,000, 2010/11 - 2020/21_Actual change","Change per 100,000, 2010/11 - 2020/21_% change"
0,Metropolitan Police,"MULTIPOLYGON (((531643.597 200802.700, 531658....",Metropolitan Police,12233.0,143.265368,12560.0,147.094991,12486.0,152.324021,13260.0,...,13708.0,154.032856,13819.0,154.362566,14685.0,165.198318,10131.0,113,-39.324021,-0.25816
1,Cumbria,"MULTIPOLYGON (((357067.497 586765.599, 357029....",Cumbria,87.0,17.473388,98.0,19.682667,128.0,25.610244,90.0,...,143.0,28.663059,162.0,32.4,203.0,41.291833,186.0,37.199107,11.588863,0.452509
2,Lancashire,"MULTIPOLYGON (((370171.702 482434.604, 370185....",Lancashire,843.0,57.269022,630.0,42.798913,674.0,46.123315,672.0,...,867.0,57.865581,1151.0,76.280734,1137.0,75.819262,1020.0,67.597076,21.473762,0.465573
3,Merseyside,"MULTIPOLYGON (((336272.101 422872.103, 336303....",Merseyside,892.0,64.121918,798.0,57.364675,606.0,43.887601,622.0,...,948.0,66.615136,1399.0,97.83901,1424.0,99.714349,1228.0,85.879531,41.991929,0.956806
4,Greater Manchester,"POLYGON ((391363.101 420849.902, 391421.401 42...",Greater Manchester,2835.0,103.735958,2361.0,86.391745,2048.0,76.264244,1737.0,...,1953.0,69.322921,3169.0,111.753712,3192.0,113.490549,3027.0,106.746657,30.482413,0.399695
5,Cheshire,"MULTIPOLYGON (((367308.201 398265.497, 367315....",Cheshire,341.0,32.813703,276.0,26.558891,266.0,25.857879,234.0,...,378.0,35.683942,374.0,35.064692,401.0,37.761819,265.0,24.844208,-1.013671,-0.039202
6,Northumbria,"MULTIPOLYGON (((398583.804 653093.703, 398566....",Northumbria,470.0,32.759462,401.0,27.950094,393.0,27.668262,375.0,...,812.0,55.746258,942.0,64.348658,794.0,54.50882,857.0,58.54213,30.873868,1.115859
7,Durham,"POLYGON ((416954.896 558226.798, 416959.398 55...",Durham,152.0,24.394158,118.0,18.93757,159.0,25.703201,157.0,...,264.0,41.673244,196.0,30.774062,165.0,26.043886,138.0,21.667554,-4.035647,-0.15701
8,North Yorkshire,"MULTIPOLYGON (((478300.699 518863.096, 478309....",North Yorkshire,225.0,27.915633,207.0,25.682382,182.0,22.778473,178.0,...,229.0,27.777778,300.0,36.201279,331.0,40.15051,343.0,41.391528,18.613055,0.817134
9,West Yorkshire,"POLYGON ((407885.502 451894.696, 407943.597 45...",West Yorkshire,1273.0,56.220465,1311.0,57.898688,1412.0,63.392296,1215.0,...,2600.0,112.059305,2807.0,120.34298,2529.0,108.998567,2200.0,94.320653,30.928357,0.487888


In [33]:
police_areas_with_kc.dtypes

pfa16nm                                                  object
geometry                                               geometry
Name                                                     object
2008/09_Number                                          float64
2008/09_Offences per 100,000 population                  object
2009/10_Number                                          float64
2009/10_Offences per 100,000 population                  object
2010/11_Number                                          float64
2010/11_Offences per 100,000 population                  object
2011/12_Number                                          float64
2011/12_Offences per 100,000 population                  object
2012/13_Number                                          float64
2012/13_Offences per 100,000 population                  object
2013/14_Number                                          float64
2013/14_Offences per 100,000 population                  object
2014/15_Number                          

In [34]:
# some blocks with missing values are represented as '-', which needs to be fixed
police_areas_with_kc[int_col+float_col] = police_areas_with_kc[int_col+float_col].replace('-', np.nan)
# and since there are missing values (np.nan) which is a floating value, we have to set all of the number to float type.
police_areas_with_kc[int_col+float_col] = police_areas_with_kc[int_col+float_col].astype('float')
police_areas_with_kc.dtypes

pfa16nm                                                  object
geometry                                               geometry
Name                                                     object
2008/09_Number                                          float64
2008/09_Offences per 100,000 population                 float64
2009/10_Number                                          float64
2009/10_Offences per 100,000 population                 float64
2010/11_Number                                          float64
2010/11_Offences per 100,000 population                 float64
2011/12_Number                                          float64
2011/12_Offences per 100,000 population                 float64
2012/13_Number                                          float64
2012/13_Offences per 100,000 population                 float64
2013/14_Number                                          float64
2013/14_Offences per 100,000 population                 float64
2014/15_Number                          

In [35]:
# do the same ununflatten procedure
unflatten_police_areas_with_kc_int = pd.melt(police_areas_with_kc[['pfa16nm', 'geometry', 'Name']+int_col], 
                                           id_vars=['pfa16nm', 'geometry', 'Name'],
                                           var_name='Year', value_vars=int_col, value_name='Number')
unflatten_police_areas_with_kc_float = pd.melt(police_areas_with_kc[['pfa16nm', 'geometry', 'Name']+float_col], 
                                             id_vars=['pfa16nm', 'geometry', 'Name'],
                                             var_name='Year', value_vars=float_col, value_name='Perc_per_pop')

unflatten_police_areas_with_kc_int['Year'] = unflatten_police_areas_with_kc_int['Year'].apply(int_col.index)
unflatten_police_areas_with_kc_float['Year'] = unflatten_police_areas_with_kc_float['Year'].apply(float_col.index)

unflatten_police_areas_with_kc = pd.merge(unflatten_police_areas_with_kc_int, unflatten_police_areas_with_kc_float, how='inner')
unflatten_police_areas_with_kc['Year'] = unflatten_police_areas_with_kc['Year'] + 2008
unflatten_police_areas_with_kc

,pfa16nm,geometry,Name,Year,Number,Perc_per_pop
0,Metropolitan Police,"MULTIPOLYGON (((531643.597 200802.700, 531658....",Metropolitan Police,2008,12233.000000,143.265368
1,Cumbria,"MULTIPOLYGON (((357067.497 586765.599, 357029....",Cumbria,2008,87.000000,17.473388
2,Lancashire,"MULTIPOLYGON (((370171.702 482434.604, 370185....",Lancashire,2008,843.000000,57.269022
3,Merseyside,"MULTIPOLYGON (((336272.101 422872.103, 336303....",Merseyside,2008,892.000000,64.121918
4,Greater Manchester,"POLYGON ((391363.101 420849.902, 391421.401 42...",Greater Manchester,2008,2835.000000,103.735958
...,...,...,...,...,...,...
597,Dorset,"MULTIPOLYGON (((377927.996 131215.099, 377954....",Dorset,2021,16.330980,0.703554
598,North Wales,"MULTIPOLYGON (((312098.398 385255.901, 312105....",North Wales,2021,17.827572,0.870390
599,Gwent,"MULTIPOLYGON (((327830.799 231005.105, 327840....",Gwent,2021,19.094354,0.918279
600,South Wales,"MULTIPOLYGON (((303435.502 214925.597, 303431....",South Wales,2021,32.495150,1.158362


In [36]:
chg_uk_pfa = unflatten_police_areas_with_kc[unflatten_police_areas_with_kc['Year']==2021].reset_index(drop=True).copy()
chg_uk_pfa.rename(columns={'Number': 'Act_chg_per_pop', 'Perc_per_pop': 'Perc_chg_per_pop'}, inplace=True)
chg_uk_pfa.drop('Year', axis=1, inplace=True)
chg_uk_pfa

,pfa16nm,geometry,Name,Act_chg_per_pop,Perc_chg_per_pop
0,Metropolitan Police,"MULTIPOLYGON (((531643.597 200802.700, 531658....",Metropolitan Police,-39.324021,-0.258160
1,Cumbria,"MULTIPOLYGON (((357067.497 586765.599, 357029....",Cumbria,11.588863,0.452509
2,Lancashire,"MULTIPOLYGON (((370171.702 482434.604, 370185....",Lancashire,21.473762,0.465573
3,Merseyside,"MULTIPOLYGON (((336272.101 422872.103, 336303....",Merseyside,41.991929,0.956806
4,Greater Manchester,"POLYGON ((391363.101 420849.902, 391421.401 42...",Greater Manchester,30.482413,0.399695
5,Cheshire,"MULTIPOLYGON (((367308.201 398265.497, 367315....",Cheshire,-1.013671,-0.039202
6,Northumbria,"MULTIPOLYGON (((398583.804 653093.703, 398566....",Northumbria,30.873868,1.115859
7,Durham,"POLYGON ((416954.896 558226.798, 416959.398 55...",Durham,-4.035647,-0.157010
8,North Yorkshire,"MULTIPOLYGON (((478300.699 518863.096, 478309....",North Yorkshire,18.613055,0.817134
9,West Yorkshire,"POLYGON ((407885.502 451894.696, 407943.597 45...",West Yorkshire,30.928357,0.487888


In [37]:
annual_uk_pfa = unflatten_police_areas_with_kc[unflatten_police_areas_with_kc['Year']!=2021].copy()
annual_uk_pfa.rename(columns={'Perc_per_pop': 'Number_per_pop'}, inplace=True)
annual_uk_pfa

,pfa16nm,geometry,Name,Year,Number,Number_per_pop
0,Metropolitan Police,"MULTIPOLYGON (((531643.597 200802.700, 531658....",Metropolitan Police,2008,12233.0,143.265368
1,Cumbria,"MULTIPOLYGON (((357067.497 586765.599, 357029....",Cumbria,2008,87.0,17.473388
2,Lancashire,"MULTIPOLYGON (((370171.702 482434.604, 370185....",Lancashire,2008,843.0,57.269022
3,Merseyside,"MULTIPOLYGON (((336272.101 422872.103, 336303....",Merseyside,2008,892.0,64.121918
4,Greater Manchester,"POLYGON ((391363.101 420849.902, 391421.401 42...",Greater Manchester,2008,2835.0,103.735958
...,...,...,...,...,...,...
554,Dorset,"MULTIPOLYGON (((377927.996 131215.099, 377954....",Dorset,2020,306.0,39.543109
555,North Wales,"MULTIPOLYGON (((312098.398 385255.901, 312105....",North Wales,2020,268.0,38.309849
556,Gwent,"MULTIPOLYGON (((327830.799 231005.105, 327840....",Gwent,2020,237.0,39.887977
557,South Wales,"MULTIPOLYGON (((303435.502 214925.597, 303431....",South Wales,2020,811.0,60.547827


In [38]:
annual_uk_pfa.to_file('police_areas_with_kc/police_areas_with_kc.shp')
chg_uk_pfa.to_file('police_areas_with_kc/chg_police_areas_with_kc.shp')

C:\Users\Yulun\AppData\Local\Temp\ipykernel_11888\4025078548.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  annual_uk_pfa.to_file('police_areas_with_kc/police_areas_with_kc.shp')
C:\Users\Yulun\AppData\Local\Temp\ipykernel_11888\4025078548.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  chg_uk_pfa.to_file('police_areas_with_kc/chg_police_areas_with_kc.shp')
